#**Installing required libraries**

In [ ]:
!pip install datasets

#**Importing the BERT Tokenizer**

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

#**Import the dataset and reframe it as per the goal of the project**

In [ ]:
import pandas as pd

df = pd.read_json('data.json')

dataset = {'title':[], 'paragraph': [], 'context': [], 'question': [], 'answer': [], 'answer_start': []}
for i in range(df['data'].shape[0]):
  paragraph_count = 0
  for j in range(len(df['data'][i]['paragraphs'])):
    for k in range(len(df['data'][i]['paragraphs'][j]['qas'])):
      for l in range(len(df['data'][i]['paragraphs'][j]['qas'][k]['answers'])):
        dataset['title'].append(df['data'][i]['title'])
        dataset['paragraph'].append(paragraph_count)
        dataset['context'].append(df['data'][i]['paragraphs'][j]['context'])
        dataset['question'].append(df['data'][i]['paragraphs'][j]['qas'][k]['question'])
        dataset['answer'].append(df['data'][i]['paragraphs'][j]['qas'][k]['answers'][l]['text'])
        dataset['answer_start'].append(df['data'][i]['paragraphs'][j]['qas'][k]['answers'][l]['answer_start'])
    paragraph_count += 1

data = pd.DataFrame.from_dict(dataset)

train = []
for i in range(len(dataset['question'])):
    train.append({
        'context': dataset['context'][i],
        'question': dataset['question'][i],
        'answer': {
            'text': dataset['answer'][i],
            'answer_start': dataset['answer_start'][i]}
        })

train[0]

#**Definition to pre-process the training data**

In [ ]:
# We need the input as follows ==> [<START> "Question here..." <SEP> "Context here..." <END>]
# And the output should be like ==> [{start-of-answer: <INT>, end-of-answer: <INT>}]

def preprocess(list_items):
  # Tokenizing the question and context
  # Pointers:
  # 1. max_length: To restrict the "question + context" length to 384
  # 2. truncation: To truncate from the "context" part off the token if length exceeds 384
  # 3. return_offsets_mapping: offset_mapping is ==> String: "This is a sentence" --> Token: [<START>, 0, 1, 2, 3, <END>] --> offset_list: [(0, 0), (0, 4), (5, 7), (8, 9), (10, 17), (18, 18)]
  inputs = tokenizer(
        list_items['question'],
        list_items["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

  # The code below is to store the start and end positions of the answers
  ##############################################################################
  sequence_ids = inputs.sequence_ids(0)

  offset_mapping = inputs.pop("offset_mapping")
  answer = list_items["answer"]
  start_positions = 0
  end_positions = 0

  start_char = answer["answer_start"]
  end_char = answer["answer_start"] + len(answer["text"])

  # Find the start and end of the context
  idx = 0
  while sequence_ids[idx] != 1:
    idx += 1
  context_start = idx
  while sequence_ids[idx] == 1:
    idx += 1
  context_end = idx - 1

  # If the answer is not fully inside the context, label it (0, 0)
  if offset_mapping[context_start][0] > end_char or offset_mapping[context_end][1] < start_char:
    start_positions = 0
    end_positions = 0
  else:
    # Otherwise it's the start and end token positions
    idx = context_start
    while idx <= context_end and offset_mapping[idx][0] <= start_char:
        idx += 1
    start_positions = (idx - 1)

    idx = context_end
    while idx >= context_start and offset_mapping[idx][1] >= end_char:
      idx -= 1
    end_positions = (idx + 1)

  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  ##############################################################################

  return inputs

#**Using 'map' to apply the definition made above to every training data**

In [ ]:
tokenized_items = list(map(preprocess, train))

#**Importing the 'model' and Preparing the training data**

In [ ]:
from transformers import TFAutoModelForQuestionAnswering
model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
from datasets import Dataset
dataset = Dataset.from_list(tokenized_items)

from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors="tf")

tf_train_set = model.prepare_tf_dataset(
    dataset,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_train_set

#**Creating the optimizer before training**

In [ ]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_items) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

#**Compiling the model created**

In [ ]:
import tensorflow as tf
model.compile(optimizer=optimizer)
model.summary()

#**Training**

In [ ]:
model.fit(x=tf_train_set, epochs=2, batch_size = 1024)

#**Trying to use numpy arrays instead of tf.datasets (Throws error)**

In [ ]:
# Whenever I try using this, I get an error as below:
# ValueError: Cannot reshape a tensor with 768 elements to shape [1,1,384,1] (384 elements) for '{{node tf_bert_for_sequence_classification_3/bert/embeddings/LayerNorm/Reshape}} =
# Reshape[T=DT_FLOAT, Tshape=DT_INT32](tf_bert_for_sequence_classification_3/bert/embeddings/LayerNorm/Reshape/ReadVariableOp,
# tf_bert_for_sequence_classification_3/bert/embeddings/LayerNorm/Reshape/shape)' with input shapes: [768], [4] and with input tensors computed as partial shapes: input1 = [1,1,384,1].

'''import numpy as np
X = np.array([
  np.array(
    [i['input_ids'] for i in tokenized_items]
    ),
  np.array(
    [i['attention_mask'] for i in tokenized_items]
    )])
X

Y = np.array([
  np.array(
    [i['start_positions'] for i in tokenized_items]
    ),
  np.array(
    [i['end_positions'] for i in tokenized_items]
    )])
Y
model.fit(x=X, y=Y, epochs=2, batch_size = 1024)'''

#**Validating the output**

In [ ]:
question1 = "How many programming languages does BLOOM support?"
question2 = "How many parameters does BLOOM have?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

inputs = tokenizer(question1, context, return_tensors="tf")
outputs = model(**inputs)
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
print(tokenizer.decode(predict_answer_tokens))

inputs = tokenizer(question2, context, return_tensors="tf")
outputs = model(**inputs)
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
print(tokenizer.decode(predict_answer_tokens))